## **Setup**

In [1]:
import tensorflow as tf
import numpy as np
import os

In [2]:
DEPTH_MULTIPLIER_PREFIX = '0'
DEPTH_MULTIPLIER = 25
IMAGE_SIZE = 128
IMAGE_SHAPE = (IMAGE_SIZE, IMAGE_SIZE)
BATCH_SIZE = 100
NUM_EPOCHS = 20

## **Load model**

In [3]:
model_name = 'mobilenet_v1_'+ DEPTH_MULTIPLIER_PREFIX + str(DEPTH_MULTIPLIER)+'_'+str(IMAGE_SIZE)+'_epochs_'+str(NUM_EPOCHS)
model_path = 'models\\' + model_name
model = tf.keras.models.load_model(model_path)

## **Initialize paths**

In [4]:
#root path
root = "datasets\\data-cifar-10"
ROOT_TRAIN_SUBDIR = "train"
ROOT_VAL_SUBDIR = "validation"
root_training_set_path = os.path.join(root, ROOT_TRAIN_SUBDIR)
root_validation_set_path = os.path.join(root, ROOT_VAL_SUBDIR)
print(root_training_set_path)
print(root_validation_set_path)

datasets\data-cifar-10\train
datasets\data-cifar-10\validation


In [5]:
OUTPUT_FOLDER_NAME = "outputs"
DATA_NAME = "CIFAR_10"

FEATURES_FOLDER = os.path.join(OUTPUT_FOLDER_NAME, "features")
FEATURES_DATA_FOLDER = os.path.join(FEATURES_FOLDER, DATA_NAME)
    
if(not os.path.exists(FEATURES_DATA_FOLDER)):
    os.makedirs(FEATURES_DATA_FOLDER)
    
MEANS_FOLDER = os.path.join(OUTPUT_FOLDER_NAME, "means")
MEANS_DATA_FOLDER = os.path.join(MEANS_FOLDER, DATA_NAME)

if(not os.path.exists(MEANS_DATA_FOLDER)):
    os.makedirs(MEANS_DATA_FOLDER)
    
DISTS_FOLDER = os.path.join(OUTPUT_FOLDER_NAME, "dists")
DISTS_DATA_FOLDER = os.path.join(DISTS_FOLDER, DATA_NAME)

if(not os.path.exists(DISTS_DATA_FOLDER)):
    os.makedirs(DISTS_DATA_FOLDER)
    
CLOSEST_CLASS_FOLDER = os.path.join(OUTPUT_FOLDER_NAME, "closest_classes")
CLOSEST_CLASS_DATA_FOLDER = os.path.join(CLOSEST_CLASS_FOLDER, DATA_NAME)

if(not os.path.exists(CLOSEST_CLASS_DATA_FOLDER)):
    os.makedirs(CLOSEST_CLASS_DATA_FOLDER)
    
LABELS_FOLDER = os.path.join(OUTPUT_FOLDER_NAME, "labels")
LABELS_DATA_FOLDER = os.path.join(LABELS_FOLDER, DATA_NAME)

if(not os.path.exists(LABELS_DATA_FOLDER)):
    os.makedirs(LABELS_DATA_FOLDER)
    
PROBABILITIES_FOLDER = os.path.join(OUTPUT_FOLDER_NAME, "probabilities")
PROBABILITIES_DATA_FOLDER = os.path.join(PROBABILITIES_FOLDER, DATA_NAME)

if(not os.path.exists(PROBABILITIES_DATA_FOLDER)):
    os.makedirs(PROBABILITIES_DATA_FOLDER)
    
RADIUS_FOLDER = os.path.join(OUTPUT_FOLDER_NAME, "radius")
RADIUS_DATA_FOLDER = os.path.join(RADIUS_FOLDER, DATA_NAME)

if(not os.path.exists(RADIUS_DATA_FOLDER)):
    os.makedirs(RADIUS_DATA_FOLDER)

## **Initialize parameters**

In [6]:
TOTAL_TRAIN_DATA_COUNT = 50000
TEMPERATURE = 100
ALPHA = 0.5

## **Init generator and class names**

In [7]:
root_image_generator = tf.keras.preprocessing.image.ImageDataGenerator(rescale=1/255)

In [8]:
root_image_data_train = root_image_generator.flow_from_directory(str(root_training_set_path), target_size=IMAGE_SHAPE, batch_size=BATCH_SIZE)
root_image_data_validation = root_image_generator.flow_from_directory(str(root_validation_set_path), target_size=IMAGE_SHAPE, batch_size=BATCH_SIZE )

Found 50000 images belonging to 10 classes.
Found 10000 images belonging to 10 classes.


In [9]:
for root_image_batch, root_label_batch in root_image_data_train:
    print("Image batch shape: ", root_image_batch.shape)
    print("Label batch shape: ", root_label_batch.shape)
    break

Image batch shape:  (100, 128, 128, 3)
Label batch shape:  (100, 10)


In [10]:
root_class_names = sorted(root_image_data_train.class_indices.items(), key=lambda pair:pair[1])
root_class_names = np.array([key.title() for key, value in root_class_names])
root_class_names

array(['Airplane', 'Automobile', 'Bird', 'Cat', 'Deer', 'Dog', 'Frog',
       'Horse', 'Ship', 'Truck'], dtype='<U10')

In [11]:
NUM_CLASSES = root_class_names.size
print("No. of classes - " + str(NUM_CLASSES))

No. of classes - 10
